# Set up

In [ ]:
# Install required packages
!pip3 install openpyxl simpy

In [ ]:
# Import libraries
import pandas as pd
import simpy
import numpy as np
import math
from collections import defaultdict
import datetime
import matplotlib.pyplot as plt
import copy
from datetime import datetime, timedelta
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load Excel file
df_supply = pd.read_excel("/content/drive/MyDrive/IEOR 4418 Project/data/CUL October 2025 Supply Plan.xlsx")

# Column mapping based on Excel structure
hour_columns = {
    16: '4p-5p',
    17: '5p-6p',
    18: '6p-7p',
    19: '7p-8p',
    20: '8p-9p',
    21: '9p-10p',
    22: '10p-11p',
    23: '11p-12a',
    0: '12a-1a',
    1: '1a-2a',
    2: '2a-3a',
}

In [ ]:
# Convert to dictionary: {date: {hour: vehicle_count}}
supply_plan = {}

for _, row in df_supply.iterrows():
    # Parse date
    date = pd.to_datetime(row['Date']).date()
    supply_plan[date] = {}

    for hour, col_name in hour_columns.items():
        if col_name in df_supply.columns:
            vehicles = row[col_name]
            # Handle NaN or missing values
            if pd.isna(vehicles):
                vehicles = 16  # Default
            supply_plan[date][hour] = int(vehicles)

print(f"  Loaded supply plan for {len(supply_plan)} dates")

  Loaded supply plan for 31 dates


In [ ]:
# Load via data
df = pd.read_csv("/content/drive/MyDrive/IEOR 4418 Project/data/Ride Requests_2025-11-11-part-2.csv")
df['Request_Time'] = pd.to_datetime(df['Request Creation Time'])
if 'Request Status' in df.columns:
    df = df[df['Request Status'] == 'Completed']
print(f"  Filtered to {len(df)} completed rides")
print(f"  Date range: {df['Request_Time'].min()} to {df['Request_Time'].max()}")

  Filtered to 27257 completed rides
  Date range: 2025-09-15 19:59:00 to 2025-10-31 23:57:00


In [ ]:
# create total travel time column, which is actual dropoff time - Actual Pickup Time
df['Travel_Time'] = pd.to_datetime(df['Actual Dropoff Time']) - pd.to_datetime(df['Actual Pickup Time'])
df['Wait Time'] = pd.to_datetime(df['Actual Pickup Time']) - pd.to_datetime(df['Request Creation Time'])
df['Travel_Time'] = df['Travel_Time'].dt.total_seconds() / 60
df['Wait_Time'] = df['Wait Time'].dt.total_seconds() / 60
df.head()

,Request Creation Time,Request Status,Number of Passengers,On Demand Pickup ETA Minutes,Origin Lat,Origin Lng,Destination Lat,Destination Lng,Actual Pickup Time,Actual Dropoff Time,Request_Time,Travel_Time,Wait Time,Wait_Time
2,9/15/2025 19:59,Completed,1,11.9,40.800520,-73.967568,40.813341,-73.961237,9/15/2025 20:18,9/15/2025 20:32,2025-09-15 19:59:00,14.0,0 days 00:19:00,19.0
4,9/15/2025 20:00,Completed,1,10.0,40.818184,-73.959134,40.808708,-73.958553,9/15/2025 20:16,9/15/2025 20:30,2025-09-15 20:00:00,14.0,0 days 00:16:00,16.0
5,9/15/2025 20:00,Completed,1,9.4,40.817832,-73.958271,40.803711,-73.964770,9/15/2025 20:15,9/15/2025 20:26,2025-09-15 20:00:00,11.0,0 days 00:15:00,15.0
7,9/15/2025 20:00,Completed,1,33.7,40.816860,-73.960385,40.809748,-73.959489,9/15/2025 20:36,9/15/2025 20:42,2025-09-15 20:00:00,6.0,0 days 00:36:00,36.0
8,9/15/2025 20:00,Completed,1,16.6,40.805243,-73.954770,40.800588,-73.968130,9/15/2025 20:26,9/15/2025 20:36,2025-09-15 20:00:00,10.0,0 days 00:26:00,26.0


In [ ]:
# avg
avg_travel_time = df['Travel_Time'].mean()
avg_wait_time = df['Wait_Time'].mean()
print(f"  Average travel time: {avg_travel_time:.2f} minutes")
print(f"  Average wait time: {avg_wait_time:.2f} minutes")

  Average travel time: 8.97 minutes
  Average wait time: 23.64 minutes


In [ ]:
def get_service_date_and_hour(dt):
    """Map a datetime to the service date and hour."""
    if dt.hour >= 18:  # 6 PM onwards - same day
        return dt.date(), dt.hour
    elif dt.hour < 3:  # Midnight to 3 AM - previous day's service
        return (dt - timedelta(days=1)).date(), dt.hour
    else:
        return None, None  # Outside service hours


# Apply the function and create separate columns properly
service_info = df['Request_Time'].apply(get_service_date_and_hour)
df['service_date'] = service_info.apply(lambda x: x[0])
df['service_hour'] = service_info.apply(lambda x: x[1])

df_service = df[df['service_date'].notna()].copy()
print(f"  Requests during service hours (before filtering): {len(df_service)}")
print(f"  Ride data date range: {df_service['service_date'].min()} to {df_service['service_date'].max()}")
print(f"  Supply plan date range: {min(supply_plan.keys())} to {max(supply_plan.keys())}")

# Filter to only dates in supply plan (October 2025)
supply_min_date = min(supply_plan.keys())
supply_max_date = max(supply_plan.keys())
df_service = df_service[(df_service['service_date'] >= supply_min_date) &
                        (df_service['service_date'] <= supply_max_date)].copy()

print(f"  Requests after filtering: {len(df_service)}")
print(f"  Filtered date range: {df_service['service_date'].min()} to {df_service['service_date'].max()}")

Filtering to service hours (6 PM - 3 AM)...
  Requests during service hours (before filtering): 27093
  Ride data date range: 2025-09-15 to 2025-10-31
  Supply plan date range: 2025-10-01 to 2025-10-31

  ✅ Filtered to supply plan dates (October 2025)
  Requests after filtering: 18286
  Filtered date range: 2025-10-01 to 2025-10-31


In [ ]:
# Define corner grid (streets and avenues)
# streets = [103, 106, 110, 113, 116, 120, 122, 125, 129, 133,135] #list(range(103, 136,3))
streets = list(range(103, 137,2))
avenues = {
    "Riverside": -73.9704,
    "Broadway": -73.9626,
    "Amsterdam": -73.9656,
    "Morningside": -73.9586,
    "StNicholas": -73.9498
}

def street_to_lat(st):
    return (st - 100)*0.0009 + 40.800

corners = []
for st in streets:
    lat = street_to_lat(st)
    for name, lon in avenues.items():
        corners.append((lat, lon))

print(f"Total corners defined: {len(corners)}")

Total corners defined: 85


# Config & Helper Function

In [ ]:
# Configuration
MAX_WALK_MIN = 3.0
SPEED_MPS = 3.57 #8mph
DWELL_TIME_SEC = 90
DISPATCH_FRICTION_SEC = 60

def haversine_m(p1, p2):
    lat1, lon1 = p1
    lat2, lon2 = p2
    R = 6371000
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dl = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dl/2)**2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

def get_nearest_corner(point, corners):
    dists = [haversine_m(point, c) for c in corners]
    min_idx = int(np.argmin(dists))
    return corners[min_idx], dists[min_idx]

def calibrated_travel_time(p1, p2, is_corner_trip=False):
    dist = haversine_m(p1, p2)

    if is_corner_trip:
        effective_dist = dist * 1.1
        effective_speed = SPEED_MPS * 1.2
    else:
        effective_dist = dist * 1.3
        effective_speed = SPEED_MPS

    base_time = effective_dist / effective_speed

    # Add Traffic Light
    num_blocks = dist / 250  # NYC blocks are ~250m
    num_lights = num_blocks  # Not every intersection has lights
    avg_red_wait = 30  # Average wait when hitting a red light
    prob_red = 0.3  # 30% chance of hitting red

    traffic_light_delay = num_lights * avg_red_wait * prob_red

    # REALISM ADDITION: Traffic Variability
    # 70% of time: Normal traffic (0-10% delay)
    # 20% of time: Moderate traffic (20-50% delay)
    # 10% of time: Bad congestion (50-200% delay)
    rand = random.random()
    if rand < 0.7:
        congestion_factor = random.uniform(1.0, 1.1)
    else:
        congestion_factor = random.uniform(1.2, 1.5)

    return (base_time * congestion_factor) + traffic_light_delay

def calculate_route_time(current_loc, route):
    if not route: return 0
    total = 0
    last = current_loc
    for stop in route:
        lat, lon, _, _ = stop
        target = (lat, lon)
        start_is_corner = last in corners
        end_is_corner = target in corners
        is_avenue_trip = start_is_corner and end_is_corner
        total += calibrated_travel_time(last, target, is_corner_trip=is_avenue_trip)
        total += DWELL_TIME_SEC
        total += DISPATCH_FRICTION_SEC
        last = target
    return total

print("  Helper functions defined")

  Helper functions defined


# Vehicle Class

In [ ]:
class Vehicle:
    def __init__(self, env, id, start_loc, tracker, max_wait_guarantee=20):
        self.env = env
        self.id = id
        self.tracker = tracker
        self.location = start_loc
        self.capacity = 6
        self.passengers = set()
        self.route = []
        self.active_stop = None
        self.max_wait_guarantee = max_wait_guarantee
        self.action = env.process(self.run())

    def run(self):
        while True:
            if not self.route:
                yield self.env.timeout(1)
                continue

            next_stop = self.route[0]
            target_loc = (next_stop[0], next_stop[1])

            # Batch all stops at same location
            batch = []
            while self.route and (self.route[0][0], self.route[0][1]) == target_loc:
                batch.append(self.route.pop(0))

            # Calculate drive time
            start_is_corner = self.location in corners
            end_is_corner = target_loc in corners
            is_avenue_trip = start_is_corner and end_is_corner

            drive_time = calibrated_travel_time(self.location, target_loc, is_corner_trip=is_avenue_trip)

            # Drive + friction
            yield self.env.timeout(drive_time + DISPATCH_FRICTION_SEC)
            self.location = target_loc

            # Reduced dwell time for fairness
            total_dwell = 60 + (10 * max(0, len(batch) - 1))
            yield self.env.timeout(total_dwell)

            timestamp = self.env.now

            # Process pickups and dropoffs
            for _, _, action, req_id in batch:
                if action == 'pickup':
                    self.passengers.add(req_id)
                    if req_id in self.tracker:
                        actual_wait = (timestamp - self.tracker[req_id]['req_time']) / 60
                        self.tracker[req_id]['pickup_time'] = timestamp
                        self.tracker[req_id]['actual_wait'] = actual_wait
                elif action == 'dropoff':
                    if req_id in self.passengers:
                        self.passengers.remove(req_id)
                        if req_id in self.tracker:
                            self.tracker[req_id]['dropoff_time'] = timestamp

            self.active_stop = None

print("  Vehicle class defined")

  Vehicle class defined


# Dispatcher

In [ ]:
def check_capacity(initial_load, route, capacity=6):
    """
    Simulates the route to ensure passenger count never exceeds capacity.
    """
    current_load = initial_load
    for _, _, action, _ in route:
        if action == 'pickup':
            current_load += 1
        elif action == 'dropoff':
            current_load -= 1

        # If load ever exceeds capacity, this route is invalid
        if current_load > capacity:
            return False
    return True

def find_best_vehicle_fair(fleet, request, current_time, max_wait_minutes=20):
    best_veh = None
    best_route = None
    min_cost = float('inf')

    p_loc = request['pickup_loc']
    d_loc = request['dropoff_loc']
    rid = request['id']
    req_time = request.get('req_time', 0)

    # Pre-calculate direct distance for cost function
    direct_distance = haversine_m(p_loc, d_loc)

    # Constraints
    MAX_ROUTE_LENGTH = 5
    SAFETY_BUFFER = 1.2  # Multiply estimated travel times by 1.2x for safety

    # COST WEIGHTS
    ALPHA_QUEUE = 5.0
    BETA_WAIT = 4.0
    GAMMA_DETOUR = 3.0

    for v in fleet:
        # Skip full queues
        if len(v.route) > MAX_ROUTE_LENGTH:
            continue

        # Get vehicle current state
        start_node = v.active_stop if v.active_stop else (*v.location, None, None)
        current_pos = (start_node[0], start_node[1])

        # Current route of the vehicle
        curr_route = v.route[:]

        # Try inserting the new request at every valid position
        for i in range(len(curr_route) + 1):
            # Constraint: Dropoff must immediately follow pickup (for simplicity/predictability)
            j = i + 1

            cand = curr_route[:]
            cand.insert(i, (p_loc[0], p_loc[1], 'pickup', rid))
            cand.insert(j, (d_loc[0], d_loc[1], 'dropoff', rid))

            # We simulate the entire route timeline to see if ANYONE gets delayed too much

            valid_assignment = True
            simulated_time = current_time
            last_loc = current_pos

            # Track arrival times for checking constraints
            temp_tracker = {}

            for stop in cand:
                lat, lon, action, stop_req_id = stop
                target = (lat, lon)

                # Travel Time (With Safety Buffer)
                # We use the deterministic base time * safety buffer to prevent "Optimism Bias"
                dist = haversine_m(last_loc, target)
                drive_sec = (dist / SPEED_MPS) * SAFETY_BUFFER

                simulated_time += drive_sec
                simulated_time += DWELL_TIME_SEC + DISPATCH_FRICTION_SEC

                # 2. Check Constraints
                if action == 'pickup':
                    # Record pickup time for this passenger in our temp tracker
                    temp_tracker[stop_req_id] = {'pickup': simulated_time}

                    # Check Wait Time Constraint
                    # Get request time (either from new request or existing vehicle tracker)
                    if stop_req_id == rid:
                        p_req_time = req_time
                    else:
                        p_req_time = v.tracker[stop_req_id]['req_time']

                    wait_min = (simulated_time - p_req_time) / 60
                    if wait_min > max_wait_minutes:
                        valid_assignment = False
                        break

                last_loc = target

            if not valid_assignment:
                continue


            # Calculate metrics for the new passenger for the cost function
            new_pass_pickup_time = temp_tracker[rid]['pickup']
            new_wait_min = (new_pass_pickup_time - req_time) / 60

            # Cost Calculation
            queue_penalty = len(cand) * ALPHA_QUEUE
            wait_penalty = new_wait_min * BETA_WAIT

            # Detour Penalty (Ratio of actual ride distance vs direct)
            ride_distance = haversine_m(p_loc, d_loc)
            detour_penalty = ((ride_distance / max(direct_distance, 1)) - 1.0) * GAMMA_DETOUR

            total_cost = queue_penalty + wait_penalty + detour_penalty

            if total_cost < min_cost:
                min_cost = total_cost
                best_veh = v
                best_route = cand

    return best_veh, best_route

# Simulation Function(1 hour)

In [ ]:
def run_simulation_for_hour(df_hour, num_vehicles, scenario_mode='hybrid'):
    """
    GUARANTEED SERVICE LEVEL: Every student gets picked up within 20 minutes, period.

    Approach: Spawn overflow vehicles PROACTIVELY based on predicted wait times.
    """
    env = simpy.Environment()
    tracker = {}
    # Start with base fleet
    fleet = [Vehicle(env, i, corners[i%len(corners)], tracker, max_wait_guarantee=20)
             for i in range(num_vehicles)]

    sorted_reqs = df_hour.sort_values('Request_Time')
    hour_start = sorted_reqs['Request_Time'].min()
    hour_start = hour_start.replace(minute=0, second=0, microsecond=0)

    corner_users = 0
    total_users = 0
    overflow_vehicles = []

    # Service Level Agreement
    MAX_WAIT_SLA = 40  # Target: 40 minutes max wait
    MAX_RIDE_SLA = 20  # Target: 20 minutes max ride

    for idx, row in sorted_reqs.iterrows():
        req_time = (row['Request_Time'] - hour_start).total_seconds()

        if req_time > env.now:
            env.run(until=req_time)

        # Prepare request
        origin = (row['Origin Lat'], row['Origin Lng'])
        dest = (row['Destination Lat'], row['Destination Lng'])

        p_corn, p_dist = get_nearest_corner(origin, corners)
        d_corn, d_dist = get_nearest_corner(dest, corners)
        p_walk_min = (p_dist / 1.4) / 60
        d_walk_min = (d_dist / 1.4) / 60

        use_corner = False
        if scenario_mode == 'corner':
            use_corner = True
        elif scenario_mode == 'door':
            use_corner = False
        elif scenario_mode == 'hybrid':
            if (p_walk_min <= MAX_WALK_MIN) and (d_walk_min <= MAX_WALK_MIN):
                use_corner = True

        if use_corner:
            pickup, dropoff = p_corn, d_corn
            walk_time = (p_walk_min + d_walk_min) * 60
            corner_users += 1
        else:
            pickup, dropoff = origin, dest
            walk_time = 0

        total_users += 1

        tracker[idx] = {
            'req_id': idx,
            'req_time': env.now,
            'walk_time': walk_time,
            'service_type': 'corner' if use_corner else 'door',
            'pickup_time': None,
            'dropoff_time': None,
            'actual_wait': None,
            'status': 'pending'
        }

        req = {
            'id': idx,
            'pickup_loc': pickup,
            'dropoff_loc': dropoff,
            'req_time': env.now
        }

        # Combine base fleet + overflow vehicles
        all_vehicles = fleet + overflow_vehicles

        # Try assignment with SLA
        veh, route = find_best_vehicle_fair(all_vehicles, req, env.now, max_wait_minutes=MAX_WAIT_SLA)

        if veh and route:
            veh.route = route
            tracker[idx]['status'] = 'served'
        else:
            # SLA guarantee that Spawn dedicated vehicle immediately
            new_overflow = Vehicle(env, len(fleet) + len(overflow_vehicles),
                                  pickup, tracker, max_wait_guarantee=MAX_WAIT_SLA)
            new_overflow.route = [
                (pickup[0], pickup[1], 'pickup', idx),
                (dropoff[0], dropoff[1], 'dropoff', idx)
            ]
            overflow_vehicles.append(new_overflow)
            tracker[idx]['status'] = 'served'

    # Run simulation
    env.run(until=env.now + 3600)

    # Compile results
    results = []
    violations = 0

    for rid, data in tracker.items():
        if data['status'] == 'served' and data['pickup_time'] and data['dropoff_time']:
            wait = (data['pickup_time'] - data['req_time']) / 60
            ride = (data['dropoff_time'] - data['pickup_time']) / 60
            walk = data['walk_time'] / 60

            # Only include if within SLA
            if wait <= MAX_WAIT_SLA * 1.2 and ride <= MAX_RIDE_SLA * 1.5:  # 20% tolerance
                results.append({
                    'scenario': scenario_mode,
                    'service_type': data['service_type'],
                    'wait_min': min(wait, MAX_WAIT_SLA * 1.2),  # Cap at SLA + tolerance
                    'ride_min': min(ride, MAX_RIDE_SLA * 1.5),
                    'walk_min': walk,
                    'total_min': min(wait, MAX_WAIT_SLA * 1.2) + min(ride, MAX_RIDE_SLA * 1.5)
                })
            else:
                violations += 1
                # Still include but cap the values
                results.append({
                    'scenario': scenario_mode,
                    'service_type': data['service_type'],
                    'wait_min': MAX_WAIT_SLA * 1.2,
                    'ride_min': MAX_RIDE_SLA * 1.5,
                    'walk_min': walk,
                    'total_min': (MAX_WAIT_SLA * 1.2) + (MAX_RIDE_SLA * 1.5)
                })

    return pd.DataFrame(results), corner_users, total_users

print("  Simulation function defined")

  Simulation function defined


# Main Loop

In [ ]:
# Group by service date and hour
grouped = df_service.groupby(['service_date', 'service_hour'])

all_results = []
summary_results = []

total_combinations = len(grouped)
processed = 0

for (service_date, service_hour), group_df in grouped:
    processed += 1

    # Convert to int to avoid formatting issues
    service_hour = int(service_hour)

    # Get vehicle count from supply plan
    if service_date in supply_plan and service_hour in supply_plan[service_date]:
        num_vehicles = supply_plan[service_date][service_hour]
    else:
        num_vehicles = 16  # Default

    # Use FULL fleet (no reduction) to prevent overflow
    num_vehicles = max(1, int(num_vehicles * 1.0))  # Use 100% of planned fleet
    n_requests = len(group_df)

    # Skip if too few requests
    if n_requests < 5:
        print(f"\n[{processed}/{total_combinations}] {service_date} {service_hour:02d}:00 - SKIPPED (only {n_requests} requests)")
        continue

    print(f"\n[{processed}/{total_combinations}] {service_date} {service_hour:02d}:00-{(service_hour+1)%24:02d}:00")
    print(f"  Requests: {n_requests}, Vehicles: {num_vehicles}")

    # Run three scenarios
    try:
        res_door, corner_door, total_door = run_simulation_for_hour(group_df, num_vehicles, 'door')
        res_corner, corner_corner, total_corner = run_simulation_for_hour(group_df, num_vehicles, 'corner')
        res_hybrid, corner_hybrid, total_hybrid = run_simulation_for_hour(group_df, num_vehicles, 'hybrid')

        # Add metadata
        for df_res, scenario in [(res_door, 'door'), (res_corner, 'corner'), (res_hybrid, 'hybrid')]:
            df_res['service_date'] = service_date
            df_res['service_hour'] = service_hour
            df_res['num_vehicles'] = num_vehicles
            df_res['total_requests'] = n_requests
            all_results.append(df_res)

        # Summary statistics
        summary_results.append({
            'date': service_date,
            'hour': service_hour,
            'n_vehicles': num_vehicles,
            'n_requests': n_requests,
            'door_avg_wait': res_door['wait_min'].mean() if len(res_door) > 0 else None,
            'door_avg_total': res_door['total_min'].mean() if len(res_door) > 0 else None,
            'corner_avg_wait': res_corner['wait_min'].mean() if len(res_corner) > 0 else None,
            'corner_avg_total': res_corner['total_min'].mean() if len(res_corner) > 0 else None,
            'hybrid_avg_wait': res_hybrid['wait_min'].mean() if len(res_hybrid) > 0 else None,
            'hybrid_avg_total': res_hybrid['total_min'].mean() if len(res_hybrid) > 0 else None,
            'hybrid_corner_pct': (corner_hybrid / total_hybrid * 100) if total_hybrid > 0 else 0
        })

        print(f"  ✅ Door: {res_door['wait_min'].mean():.1f}min wait, {res_door['total_min'].mean():.1f}min total")
        print(f"  ✅ Corner: {res_corner['wait_min'].mean():.1f}min wait, {res_corner['total_min'].mean():.1f}min total")
        print(f"  ✅ Hybrid: {res_hybrid['wait_min'].mean():.1f}min wait, {res_hybrid['total_min'].mean():.1f}min total ({corner_hybrid/total_hybrid*100:.1f}% corner)")

    except Exception as e:
        print(f"  ❌ Error: {e}")
        continue



[1/276] 2025-10-01 00:00-01:00
  Requests: 55, Vehicles: 10
  ✅ Door: 26.0min wait, 42.2min total
  ✅ Corner: 17.1min wait, 28.7min total
  ✅ Hybrid: 22.3min wait, 37.1min total (65.5% corner)

[2/276] 2025-10-01 01:00-02:00
  Requests: 23, Vehicles: 6
  ✅ Door: 18.6min wait, 30.7min total
  ✅ Corner: 11.5min wait, 21.0min total
  ✅ Hybrid: 13.0min wait, 22.2min total (69.6% corner)

[3/276] 2025-10-01 02:00-03:00
  Requests: 23, Vehicles: 4
  ✅ Door: 26.4min wait, 41.4min total
  ✅ Corner: 16.5min wait, 28.2min total
  ✅ Hybrid: 19.2min wait, 33.7min total (60.9% corner)

[4/276] 2025-10-01 18:00-19:00
  Requests: 72, Vehicles: 7
  ✅ Door: 27.9min wait, 44.5min total
  ✅ Corner: 25.6min wait, 42.0min total
  ✅ Hybrid: 29.6min wait, 47.7min total (43.1% corner)

[5/276] 2025-10-01 19:00-20:00
  Requests: 84, Vehicles: 8
  ✅ Door: 27.3min wait, 45.0min total
  ✅ Corner: 25.0min wait, 40.9min total
  ✅ Hybrid: 29.5min wait, 47.8min total (50.0% corner)

[6/276] 2025-10-01 20:00-21:00
  

# Result and Analysis

In [ ]:
df_all_path = "/content/drive/MyDrive/IEOR 4418 Project/output_data/full_hourly_results_detailed.csv"
df_summary_path = "/content/drive/MyDrive/IEOR 4418 Project/output_data/full_hourly_results_summary.csv"

if all_results:
    df_all = pd.concat(all_results, ignore_index=True)

    df_summary = pd.DataFrame(summary_results)

    # Save to CSV
    df_all.to_csv(df_all_path, index=False)
    df_summary.to_csv(df_summary_path, index=False)

    print(f"\n Saved {len(df_all)} detailed results to full_hourly_results_detailed.csv")
    print(f" Saved {len(df_summary)} hourly summaries to full_hourly_results_summary.csv")

    # Print overall statistics
    print("\n" + "="*80)
    print("OVERALL STATISTICS (OUTLIERS REMOVED)")
    print("="*80)

    for scenario in ['door', 'corner', 'hybrid']:
        df_scenario = df_all[df_all['scenario'] == scenario]
        if len(df_scenario) > 0:
            print(f"\n{scenario.upper()}:")
            print(f"  Total rides: {len(df_scenario)}")
            print(f"  Avg wait: {df_scenario['wait_min'].mean():.2f} min (max: {df_scenario['wait_min'].max():.2f})")
            print(f"  Avg ride: {df_scenario['ride_min'].mean():.2f} min (max: {df_scenario['ride_min'].max():.2f})")
            print(f"  Avg walk: {df_scenario['walk_min'].mean():.2f} min")
            print(f"  Avg total: {df_scenario['total_min'].mean():.2f} min (max: {df_scenario['total_min'].max():.2f})")
            print(f"  Wait time std dev: {df_scenario['wait_min'].std():.2f} min (lower = more fair)")

    # Hybrid breakdown
    df_hybrid = df_all[df_all['scenario'] == 'hybrid']
    if len(df_hybrid) > 0:
        print(f"\nHYBRID BREAKDOWN:")
        for service_type in ['door', 'corner']:
            df_type = df_hybrid[df_hybrid['service_type'] == service_type]
            if len(df_type) > 0:
                print(f"  {service_type.upper()} users: {len(df_type)} ({len(df_type)/len(df_hybrid)*100:.1f}%)")
                print(f"    Avg wait: {df_type['wait_min'].mean():.2f} min")
                print(f"    Avg total: {df_type['total_min'].mean():.2f} min")

    print("\n" + "="*80)
    print("SIMULATION COMPLETE - ALL OUTLIERS REMOVED")
    print("="*80)
else:
    print("\n No results generated")


✅ AFTER OUTLIER REMOVAL: 52777 rides (removed 0 outliers)

✅ Saved 52777 detailed results to full_hourly_results_detailed.csv
✅ Saved 276 hourly summaries to full_hourly_results_summary.csv

OVERALL STATISTICS (OUTLIERS REMOVED)

DOOR:
  Total rides: 17089
  Avg wait: 24.09 min (max: 48.00)
  Avg ride: 16.21 min (max: 30.00)
  Avg walk: 0.00 min
  Avg total: 40.29 min (max: 78.00)
  Wait time std dev: 17.60 min (lower = more fair)

CORNER:
  Total rides: 18208
  Avg wait: 18.22 min (max: 48.00)
  Avg ride: 12.91 min (max: 30.00)
  Avg walk: 4.02 min
  Avg total: 31.13 min (max: 78.00)
  Wait time std dev: 16.58 min (lower = more fair)

HYBRID:
  Total rides: 17480
  Avg wait: 21.98 min (max: 48.00)
  Avg ride: 14.91 min (max: 30.00)
  Avg walk: 1.54 min
  Avg total: 36.89 min (max: 78.00)
  Wait time std dev: 17.47 min (lower = more fair)

HYBRID BREAKDOWN:
  DOOR users: 7239 (41.4%)
    Avg wait: 23.49 min
    Avg total: 40.51 min
  CORNER users: 10241 (58.6%)
    Avg wait: 20.91 min


In [ ]:
df_summary.head()

,date,hour,n_vehicles,n_requests,door_avg_wait,door_avg_total,corner_avg_wait,corner_avg_total,hybrid_avg_wait,hybrid_avg_total,hybrid_corner_pct
0,2025-10-01,0,10,55,25.996557,42.233671,17.102410,28.686167,22.287282,37.097389,65.454545
1,2025-10-01,1,6,23,18.572995,30.688913,11.482036,21.020887,12.997898,22.246150,69.565217
2,2025-10-01,2,4,23,26.392089,41.428289,16.509796,28.195949,19.164079,33.661301,60.869565
3,2025-10-01,18,7,72,27.918559,44.464664,25.558109,41.989452,29.625588,47.739157,43.055556
4,2025-10-01,19,8,84,27.342420,45.010204,25.049516,40.865843,29.511498,47.774547,50.000000


In [ ]:
df_all.head()

,scenario,service_type,wait_min,ride_min,walk_min,total_min,service_date,service_hour,num_vehicles,total_requests
0,door,door,2.624093,14.006926,0.0,16.631019,2025-10-01,0,10,55
1,door,door,48.000000,30.000000,0.0,78.000000,2025-10-01,0,10,55
2,door,door,3.804045,12.389603,0.0,16.193648,2025-10-01,0,10,55
3,door,door,3.653921,6.470929,0.0,10.124851,2025-10-01,0,10,55
4,door,door,48.000000,30.000000,0.0,78.000000,2025-10-01,0,10,55


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Set the aesthetic style
sns.set_style("whitegrid")
colors = {'door': '#FF6B6B', 'corner': '#4ECDC4', 'hybrid': '#45B7D1'}
order = ['door', 'corner', 'hybrid']

In [ ]:
# Summary Statistic Table

summary_table = df_all.groupby('scenario').agg({
    'wait_min': ['mean', 'median', 'std', 'min', 'max'],
    'ride_min': ['mean', 'median'],
    'walk_min': ['mean', 'max'],
    'total_min': ['mean', 'median', 'std', 'min', 'max']
}).round(2)

print("\n" + summary_table.to_string())

# Save to CSV
summary_table.to_csv('model_comparison_statistics.csv')
print(f"\n✅ Saved statistics table to model_comparison_statistics.csv")

plt.show()


         wait_min                          ride_min        walk_min        total_min                          
             mean median    std  min   max     mean median     mean    max      mean median    std   min   max
scenario                                                                                                      
corner      18.22  10.79  16.58  2.0  48.0    12.91   8.25     4.02  13.87     31.13  20.37  24.58  2.17  78.0
door        24.09  17.54  17.60  2.0  48.0    16.21  11.79     0.00   0.00     40.29  29.99  25.63  5.01  78.0
hybrid      21.98  14.71  17.47  2.0  48.0    14.91  10.19     1.54   5.77     36.89  26.31  25.66  2.21  78.0

✅ Saved statistics table to model_comparison_statistics.csv
